# 隐式马尔可夫模型基础

## 摘要

我们如何将机器学习应用在基于时间序列的观测数据上？举个例子，比如我们可能想要从某人演讲的音频记录中得到他使用的词语序列；或者根据词性标注对一系列单词做注解等等。在本文中，我们将对马尔可夫模型（在时间序列上的状态的形式化推理）以及隐式马尔可夫模型（从观测数据的序列中找回原来的状态序列）的概念从数学方面做一个全面的介绍。本文的最后一节包含了本文中某些观点的来源并提供深入学习的材料。

## 1. 马尔可夫模型

对于给定的状态集$S=\left\{s_1,s_2,\cdots,s_{\lvert S\rvert}\right\}$，我们可以沿时间序列观测到一些列数据$\vec z\in S^T$。举个例子，一个从天气系统中得到的状态集$S=\left\{sun,cloud,rain\right\}$，集合的大小为$\lvert S\rvert=3$，我们也在天气系统中得到一组观测数据$\left\{z_1=s_{sun},z_2=s_{cloud},z_3=s_{cloud},z_4=s_{rain},z_5=s_{cloud}\right\}$，观测数据集大小为$T=5$。

天气例子的的观测状态向我们展示了一个在时间上的随机过程的输出。如果不做进一步假设，$t$时刻的状态$s_j$可能是一个关于任意多个变量的函数，其中包括了从时刻$1$到$t-1$的所有状态、以及一些我们甚至都没有建模的变量。不过，我们可以做出两个能够使时间序列更已处理的**马尔可夫假设（Markov Assumptions）**。

**限制时域假设（Limited Horizon Assumption）**指的是到达$t$时刻状态的概率只与$t-1$时刻的状态有关。这条假设的直观理解就是$t$时刻的状态“足以概括”之前所有状态，并可以合理的预测未来的状态，形式化的表示为：
$$
P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1\right)=P\left(z_t\mid z_{t-1}\right)
$$

**平稳过程假设（Stationary Process Assumption）**指的是在当前状态下，下个状态的条件分布不随时间变化而改变，形式化的表示为：
$$
P\left(z_t\mid z_{t-1}\right)=P\left(z_2\mid z_1\right);\ t\in2\cdots T
$$

按照习惯，我们也会假设初始状态和初始观测数据满足$z_0\equiv s_0$，$s_0$表示在$0$时刻的状态的初始概率分布。这个习惯可以使我们将“观测第一个真实状态$z_1$的先验概率”的观点通过$P\left(z_1\mid z_0\right)$表达出来。因为我们定义对于任意状态序列都有$z_0=s_0$，所以$P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1\right)=P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1,z_0\right)$。（其他HMM的表示方法有时通过向量$\pi\in\mathbb R^{\lvert S\rvert}$表达其先验的观点。）

我们定义一个状态转换矩阵$\displaystyle A\in\mathbb R^{(\lvert S\rvert+1)\times(\lvert S\rvert+1)}$来参数化这些转换，$A_{ij}$就表示在任意时刻$t$下从状态$i$转换到状态$j$的概率。（可以参考[线性代数中马尔科夫矩阵的例子](http://nbviewer.jupyter.org/github/zlotus/notes-linear-algebra/blob/master/chapter24.ipynb)）在天气系统的例子中，就可以定义如下的状态转换矩阵：
$$
A=\begin{matrix}
&s_0&s_{sun}&s_{cloud}&s_{rain}\\
s_0&0&.33&.33&.33\\
s_{cloud}&0&.2&.6&.2\\
s_{rain}&0&.1&.2&.7
\end{matrix}
$$

容易观察到这些数字（虽然是编造的）表示的直观的天气的自相关性：如果当前是晴天，则天气倾向于保持晴朗；如果是多云则倾向于保持多云，等等。这种模式经常在马尔可夫模型中出现，体现在状态转移矩阵中就是对角线元素远大于其他元素。在本例中，初始状态$s_0$是转换到这三个天气状态上的均匀概率分布。

### 1.1 马尔可夫模型的两个问题

结合马尔可夫假设和状态转换参数$A$，我们可以回答在马尔科夫链中关于状态序列的两个基本问题：状态$\vec z$对应的一个特定序列的概率是什么？对于观测到的序列$\vec z$，如何估计模型中的参数$A$的才能最大化其似然性？

#### 1.1.1 状态序列的概率

我们可以利用概率论中的[链式法则](https://en.wikipedia.org/wiki/Chain_rule_(probability))状态$\vec z$对应的一个特定序列的概率：
$$
\begin{align}
P\left(\vec z\right)&=P\left(z_t,z_{t-1},\cdots,z_1;A\right)\\
&=P\left(z_t,z_{t-1},\cdots,z_1,z_0;A\right)\\
&=P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1;A\right)P\left(z_{t-1}\mid z_{t-2},\cdots,z_1;A\right)\cdots P(z_1\mid z_0;A)\\
&=P\left(z_t\mid z_{t-1};A\right)P\left(z_{t-1}\mid z_{t-2};A\right)\cdots P\left(z_2\mid z_1;A\right)P\left(z_1\mid z_0;A\right)\\
&=\prod_{t=1}^TP\left(z_{t}\mid z_{t-1};A\right)\\
&=\prod_{t=1}^TA_{z_{t-1}\ z_t}
\end{align}
$$

式中的第二步，我们根据前面关于$z_0$的定义，将$z_0$引入联合分布。第三步是由对联合分布应用链式法则，或重复迭代贝叶斯发展推导出的。第四步使用了马尔可夫模型的限制时域假设。而最后一步是将这些乘积项用其对应在状态转换矩阵$A$中的元素表示出来。

我们来计算一下天气系统例子中的序列：如果要计算序列$P\left(z_1=s_{sun},z_2=s_{cloud},z_3=s_{rain},z_4=s_{rain},z_5=s_{cloud}\right)$的概率，则可以分解为$P\left(s_{sun}\mid s_0\right)P\left(s_{cloud}\mid s_{sun}\right)P\left(s_{rain}\mid s_{cloud}\right)P\left(s_{rain}\mid s_{rain}\right)P\left(s_{cloud}\mid s_{rain}\right)=.33\times.1\times.2\times.7\times.2$。

#### 1.1.2 参数的最大似然估计

从学习的角度，我们想要知道能够“最大化观测到的序列$\vec z$的对数似然性”的参数$A$的取值。这相当于比较“从晴到阴”与“从晴到晴”等状态的似然性，以使得某个观测序列更有可能发生。我们定义马尔可夫模型的对数似然函数：

$$
\begin{align}
l(A)&=\log P\left(\vec z;A\right)\\
&=\log\prod_{t=1}^TA_{z_{t-1}\ z_t}\\
&=\sum_{t=1}^T\log A_{z_{t-1}\ z_t}\\
&=\sum_{i=1}^{\lvert S\rvert}\sum_{j=1}^{\lvert S\rvert}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}\log A_{ij}
\end{align}
$$

我们在式中的最后一步使用了指示函数（其参数成立时函数值取$1$，否则取$0$）来从状态转移矩阵$A$中选出从时间序列中观测到的每一步状态转移对应的概率。在解这个优化问题时要注意，解出的参数$A$必须满足一个状态转移矩阵的要求——于每个状态$i$对应的分布，其概率之和必须为$1$；矩阵的所有元素均为非负数。我们可以使用拉格朗日乘数法来求解这个优化问题：

$$
\begin{align}
\max_A&\ l(A)\\
\mathrm{s.t.}&\ \sum_{j=1}^{\lvert S\rvert}A_{ij}=1,\ i=1,\cdots,\lvert S\rvert\\
&\ A_{ij}\geq0,\ i,j=1,\cdots,\lvert S\rvert
\end{align}
$$

这个带有约束条件的优化问题可以使用拉格朗日乘数法直接求出其解析解。我们会将等式约束条件（即第一个约束条件）引入拉格朗日算子，而不等式约束条件则可以方向的忽略——因为最优解中的$A_{ij}$的元素都是非负的。于是，构造拉格朗日算子：

$$
\mathcal L(A,\alpha)=\sum_{i=1}^{\lvert S\rvert}\sum_{j=1}^{\lvert S\rvert}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}\log A_{ij}+\sum_{i=1}^{\lvert S\rvert}\alpha_i\left(1-\sum_{j=1}^{\lvert S\rvert}A_{ij}\right)
$$

对拉格朗日算子求关于$A_{ij}$的偏导，并将偏导置为$0$得到：

$$
\begin{align}
\frac{\partial\mathcal L(A,\alpha)}{\partial A_{ij}}&=\frac{\partial}{\partial A_{ij}}\left(\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}\log A_{ij}\right)+\frac{\partial}{\partial A_{ij}}\alpha_i\left(1-\sum_{j=1}^{\lvert S\rvert}A_{ij}\right)\\
&=\frac{1}{A_{ij}}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}-\alpha_{i}\equiv0\\
&\Downarrow\\
A_{ij}&=\frac{1}{\alpha_i}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}
\end{align}
$$

再求关于$\alpha$的偏导，并将偏导置为$0$且将上式代入得到：

$$
\begin{align}
\frac{\partial\mathcal L(A,\alpha)}{\partial\alpha_i}&=1-\sum_{j=1}^{\lvert S\rvert}A_{ij}\\
&=1-\sum_{j=1}^{\lvert S\rvert}\frac{1}{\alpha_i}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}\equiv0\\
\Downarrow\\
\alpha_i&=\sum_{j=1}^{\lvert S\rvert}\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}\\
&=\sum_{t=1}^T1\left\{z_{t-1}=s_i\right\}
\end{align}
$$

再将得到的$\alpha_i$代回前面$A_{ij}$的表达式，就能得到参数$\hat A_{ij}$的最大似然估计：

$$
\hat A_{ij}=\frac{\sum_{t=1}^T1\left\{z_{t-1}=s_i\land z_{t}=s_j\right\}}{\sum_{t=1}^T1\left\{z_{t-1}=s_i\right\}}
$$

这个式子的意义非常直观——系统从状态$i$转移到状态$j$的最大似然概率为“系统从状态$i$转移到状态$j$的次数”除以“系统处于状态$i$的次数”。换句话说，最大似然参数应为系统处于状态$i$时转移到状态$j$的概率。

\left(\right)
\begin{align}\end{align}
\left\{\right\}